# Question Answering su modelli in locale con Ollama - Nomic - FAISS - Gemma2

### Ollama

<img src="img/ollama.png" width=200>    
    
Ollama consente di eseguire modelli linguistici di grandi dimensioni open source, come Llama 3 o Gemma, localmente.    
Ollama organizza i pesi del modello, la sua configurazione e i dati necessari a farlo funzionare in un unico pacchetto, definito da un *Modelfile*; si occupa anche dell'installazione e della configurazione, incluso l'utilizzo delle eventuali GPU disponibili, semplificando l'utilizzo di LLMs in locale.
    
E' possibile installare Ollama direttamente dal sito web ufficiale https://ollama.com/download.    
    
Per scaricare un modello, ad esempio Gemma di Google, utilizzare la seguente sintassi nel terminale:    
**ollama pull gemma**    
    
Possiamo sfruttare Ollama per scaricare anche degli Embeddings preaddestrati:    
**ollama pull nomic-embed-text**    
    
Per lanciare Ollama in esecuzione:    
**ollama serve**    
    
<hr>    
    
#### Caratteristiche principali di Ollama
* **Esecuzione locale**
    * Privacy e Sicurezza: siccome i dati vengono elaborati localmente, gli utenti mantengono il pieno controllo delle informazioni, riducendo i rischi legati alla privacy associati all'uso di servizi online.
    * Accesso Offline: Ollama permette l'uso dei modelli senza necessità di connessione a Internet, il che è particolarmente utile in contesti con scarsa connettività.
    * Costi: Utilizzando Ollama, gli utenti possono evitare le spese associate ai servizi di AI basati su cloud, poiché possono sfruttare il proprio hardware per eseguire i modelli.
* **Facilità d'uso**
    * Interfaccia Semplice: Ollama offre un'API intuitiva che semplifica l'interazione con i modelli linguistici. Gli utenti possono facilmente scaricare, eseguire e gestire diversi LLM tramite comandi semplici.
    * Supporto per Modelli Vari: Il progetto supporta una gamma di modelli open source come Llama 2, Mistral e Vicuna, consentendo agli sviluppatori di scegliere il modello più adatto alle loro esigenze specifiche.
    * Integrazione: Ollama fornisce un'API REST che facilita l'integrazione dei modelli in altre applicazioni software, permettendo agli sviluppatori di utilizzare i LLM in progetti complessi e automatizzati.
* **Popolarità**
    * Controllo e Personalizzazione: Gli utenti possono importare modelli personalizzati e modificare le impostazioni predefinite, rendendo possibile l'adattamento dei modelli alle specifiche applicazioni.
    * Ampia Comunità e Risorse: Essendo open source, Ollama beneficia di una comunità attiva che contribuisce al suo sviluppo e alla creazione di risorse utili, come tutorial e guide.
    * Versatilità: La capacità di eseguire vari modelli su hardware comune consente a una vasta gamma di utenti—da ricercatori a professionisti—di sfruttare le potenzialità dei LLM senza investimenti significativi in infrastrutture cloud.

# Google Gemma

<img src="https://i2.res.24o.it/images2010/Editrice/ILSOLE24ORE/NOVA24/2024/02/23/Nova24/ImmaginiWeb/Ritagli/Gemma_blog_2-kfUG--1020x533@IlSole24Ore-Web.jpg?r=1080x566" width=400>  
  
Google Gemma è una famiglia di modelli aperti e leggeri, basati sulla tecnologia usata per creare i modelli Google Gemini.  
Sono modelli decoder-only e sono adatti per una varietà di task che vanno dalla generazione di testo, tra cui la risposta alle domande e il riepilogo fino al ragionamento. Le loro dimensioni relativamente ridotte ne consentono l’implementazione in ambienti con risorse limitate come laptop, desktop o la propria infrastruttura cloud, democratizzando l’accesso a modelli di intelligenza artificiale all’avanguardia.  
Le prima versioni della famiglia Gemma si aspettano input testuali sotto forma di domanda o di testo da riassumere e sono addestreato a rispondere in lingua inglese.     
([elenco versioni Google Gemma](https://ai.google.dev/gemma/docs/releases))      
Addestrati su un set di dati di 6 trilioni di token da web, codice e testi matematici, questi modelli acquisiscono una vasta conoscenza di stili linguistici, vocaboli, sintassi di programmazione e ragionamento logico, risultando potenti strumenti per diverse attività e formati di testo.  
Gemma è stata addestrata utilizzando i processori TPU custom di Google per sfruttare al meglio prestazioni, memoria, scalabilità ed efficienza in termini di costi, in linea con l'impegno di Google per la sostenibilità.


In [1]:
# caricamento del modello

from langchain_ollama import OllamaLLM
model = OllamaLLM(model="gemma3:12b")

In [2]:
# utilizzo del modello in locale

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser 

prompt = ChatPromptTemplate.from_template("Genera una descrizione buffa e divertente sul seguente argomento: {argomento}. Rispondi in italiano. Crea una risposta breve.")

output_parser = StrOutputParser()

chain = prompt | model | output_parser

answer=chain.invoke({"argomento": "Deep Learning"})

print(answer)

Deep Learning: È come insegnare al tuo computer a fare le facce buffe guardando miliardi di foto di gatti. Dopo un po' lo fa anche da solo, ma a volte ti fa sorrisi inquietanti che ti fanno dubitare della sanità mentale della macchina. Divertente, no? (Speriamo non troppo.)



### RAG

In [38]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# RecursiveUrlLoader è un loader che ci permette di caricare in modo ricorsivo tutte le pagine web collegate ad una pagina web iniziale
# max_depth indica la profondità massima di ricorsione
# extractor è la funzione che si occupa di estrarre e parserizzare il testo dalla pagina web
url = "https://datamasters.it/"
loader = RecursiveUrlLoader(
    url=url, max_depth=2, extractor=lambda x: Soup(x, "html.parser").text.replace("\n", " ").replace("\t", " ").strip()
)
docs = loader.load()

In [39]:
len(docs)

44

In [40]:
docs[0].metadata

{'source': 'https://datamasters.it/',
 'content_type': 'text/html',
 'title': 'I migliori Corsi di Intelligenza Artificiale | Data Masters',
 'description': 'I migliori corsi online di intelligenza artificiale, machine learning e data science. Formazione pratica, docenti esperti, lezioni on-demand e live. Entra ora!',
 'language': 'it-IT'}

In [41]:
len(docs[0].page_content)

11963

In [42]:
docs[0].page_content

'I migliori Corsi di Intelligenza Artificiale | Data Masters                                                                IMPARA A CREARE AGENTI AI\xa0 SCONTO 50% IN SCADENZA Scopri di piùDataMastersCatalogoCorsiPercorsi di CarrieraNon sai da dove partire? Compila il test di orientamento e ricevi consigli personalizzati Read morePrincipianteIntermedioAvanzatoCategorieCorsi di Generative AICorsi di Data AnalysisCorsi di Data ScienceCorsi di Machine LearningCorsi di Programmazione in PythonTutti i corsiPer le aziendeCommunityCommunity HubBlogChallengesWebinarsAI e Data Skill Report 2025Chi siamoLavora con noiContattiAccediFai crescere la tua carriera. Inizia davvero a usare dati e AI Formazione pratica e flessibile su Intelligenza Artificiale, Data Science, Machine Learning e Generative AI.Corsi live e on-demand progettati per far evolvere le tue competenze, con la guida di docenti esperti. Scopri il nostro catalogo  Trustpilot   Formiamo i professionisti delle migliori aziende al mond

#### Spliting Text

In [43]:
from langchain_text_splitters import RecursiveCharacterTextSplitter 

text_splitter = RecursiveCharacterTextSplitter(chunk_size=850, chunk_overlap=350)
splits = text_splitter.split_documents(docs)

In [44]:
print("Numero di chunks : ",len(splits))

Numero di chunks :  1391


In [45]:
print("Metadata :",splits[0].metadata )
print ("Dimensione Chunk n.1:",len (splits[0].page_content), " caratteri")
print ("Contenuto Chunk n.1 : \n")
print(splits[0].page_content)

Metadata : {'source': 'https://datamasters.it/', 'content_type': 'text/html', 'title': 'I migliori Corsi di Intelligenza Artificiale | Data Masters', 'description': 'I migliori corsi online di intelligenza artificiale, machine learning e data science. Formazione pratica, docenti esperti, lezioni on-demand e live. Entra ora!', 'language': 'it-IT'}
Dimensione Chunk n.1: 850  caratteri
Contenuto Chunk n.1 : 

I migliori Corsi di Intelligenza Artificiale | Data Masters                                                                IMPARA A CREARE AGENTI AI  SCONTO 50% IN SCADENZA Scopri di piùDataMastersCatalogoCorsiPercorsi di CarrieraNon sai da dove partire? Compila il test di orientamento e ricevi consigli personalizzati Read morePrincipianteIntermedioAvanzatoCategorieCorsi di Generative AICorsi di Data AnalysisCorsi di Data ScienceCorsi di Machine LearningCorsi di Programmazione in PythonTutti i corsiPer le aziendeCommunityCommunity HubBlogChallengesWebinarsAI e Data Skill Report 2025C

nomic-embed-text è un piccolo modello di embedding il locale che possiamo utilizzare sfruttando direttamente Ollama

In [46]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="nomic-embed-text"
)

### Faiss Vector Database

<img src="https://engineering.fb.com/wp-content/uploads/2017/03/faiss_logo.png?w=250">

Faiss è una libreria per il clustering e la ricerca efficiente su vettori densi.    
Contiene algoritmi che cercano in serie di vettori di qualsiasi dimensione, fino a quelli che eventualmente non rientrano nella RAM.    
    
Rispetto a ChromaDB ci mette a disposizione un po' più di accelerazione hardware e feature varie, rendendolo un ottimo progetto per progetti medio/grandi, ma nel caso di aziende che fatturano più di una certa soglia (circa 100,000$ l'anno), può essere utilizzato solo lato pagamento di licenza a Meta Research, mentre è liberamente utilizzabile per progetti non commerciali

Pagina del progetto su GitHub -> [https://github.com/facebookresearch/faiss](https://github.com/facebookresearch/faiss)        
    
<hr>    
     

Lista VectorStore disponibili con LangChain:    
* [https://python.langchain.com/docs/integrations/vectorstores/](https://python.langchain.com/docs/integrations/vectorstores/)

In [48]:
# Creazione di un database vettoriale Faiss (Facebook Research)

import os
from langchain_community.vectorstores import FAISS  # pip install faiss-gpu 
                                                    # oppure pip install faiss-cpu

if os.path.exists("data/backup_dm"):
    # Carica indice FAISS precedentemente costruito
    faiss_index = FAISS.load_local(
        "data/backup_dm",
        embeddings,
        allow_dangerous_deserialization=True # necessario per caricare indici creati con versioni precedenti di Faiss
    )
    print("Backup caricato.")
else:
    # Crea indice FAISS dei chunk e li salva su disco
    faiss_index = FAISS.from_documents(
        splits,
        embeddings
    )
    faiss_index.save_local("data/backup_dm")
    print("Backup creato e salvato.")

Backup caricato.


In [49]:
# come cercare nel VectorStore

docs = faiss_index.similarity_search("Starter Kit", k=3)

for doc in docs:
    print ("documento:", doc.metadata["source"]) 
    print (doc.page_content)
    print ("_" * 80, "\n")

documento: https://datamasters.it/
solo […]Leggi tuttoLeggi tuttoWEBINAR01OTT 18:30Gemini e Generative AI: come utilizzare strategicamente l'AI di GoogleUnisciti a noi per questo webinar esclusivo con una demo pratica, in cui esploreremo le ultime novità di Gemini, il modello di intelligenza artificiale firmato Google, sempre più al centro delle applicazioni professionali in ambito AI.Iscriviti oraIscriviti oraNEWS01OTTData Masters entra nel Google for Startups Cloud ProgramUn nuovo traguardo per Data Masters Siamo orgogliosi di annunciare che Data Masters è stata accettata nel Google for Startups Cloud Program, il programma che supporta le startup più innovative […]Leggi tuttoLeggi tuttoNEWS01OTTCos'è Warmwind OS e come rivoluzionerà il mondo dei sistemi operativiLa tecnologia dei sistemi operativi sta per subire una rivoluzione, grazie all’arrivo di Warmwind OS, il
________________________________________________________________________________ 

documento: https://datamasters.it/
di

In [50]:
# usiamo il database Faiss creato poco fa come
# elemento "retriever" per la nostra catena di
# generazione basata su RAG

retriever = faiss_index.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 7}
)

In [51]:
from langchain import hub

# recupero di un prompt standard
prompt = hub.pull("rlm/rag-prompt")

# Una volta recuperato il prompt, possiamo modificarlo a nostro piacimento
# in questo contesto lo abbiamo rimesso praticamente identico a quello di default
# ma volendo avremmo potuto cambiarlo a nostro piacimento, a patto di mantenere le variabili di input {question} e {context}
prompt.messages[0].prompt.template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"

/home/simone/aaamc/venv/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [52]:
from langchain_core.runnables import RunnablePassthrough


def format_documents(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
        {"context": retriever | format_documents, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
)

In [53]:
rag_chain.invoke("quale corso mi consigli per partire da zero?")

"DataMasters offre corsi adatti a chi parte da zero, in particolare nella data science. Alcuni utenti hanno apprezzato i corsi su ChatGPT e Copilot, mentre altri hanno trovato utili i corsi intensivi sull'IA generativa. DataMasters sembra essere una buona scelta per chi vuole iniziare a studiare."

In [34]:
rag_chain.invoke("quali servizi offre Data Masters alle aziende?")

'Data Masters offre alle aziende formazione pratica e flessibile su Intelligenza Artificiale, Data Science, Machine Learning e Generative AI. I corsi sono progettati per far evolvere le competenze dei dipendenti con la guida di docenti esperti e possono essere fruiti in modalità on-demand e live. Data Masters offre anche percorsi di carriera personalizzati per aiutare i dipendenti a raggiungere i loro obiettivi professionali.'

### Retrieval-Chain standard con memoria conversazionale in LangChain

Utilizzo di una memoria sequenziale per gestire il contesto su conversazioni e di una sintesi dei messaggi nel contesto per generare la risposta

In [55]:
# ci crea un retriever in grado di tenere conto della storia della conversazione
from langchain.chains import create_history_aware_retriever

# ci crea una chain di RAG completa
from langchain.chains import create_retrieval_chain

# chain standard per combinare documenti
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# recupero della sola memoria conversazionale utile a rispondere alla domanda attuale
testo_prompt_contestualizzato = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

prompt_contestualizzato = ChatPromptTemplate.from_messages(
    [
        ("system", testo_prompt_contestualizzato),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# create_history_aware_retriever è una chain standard
# progettata per rendere il retriever "consapevole" 
# della storia della conversazione
history_aware_retriever = create_history_aware_retriever(
    model, retriever, prompt_contestualizzato
)

system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\
Use italian language.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# `create_stuff_documents_chain` è una chain che si occupa di combinare 
# i documenti recuperati con la query per generare una risposta. 
# Il nome "stuff" si riferisce alla strategia più semplice di 
# combinazione dei documenti:
# - prende tutti i documenti recuperati
# - li "infila" (stuffs) nel prompt insieme alla domanda
# - fa generare una risposta al LLM
question_answer_chain = create_stuff_documents_chain(model, qa_prompt)

# `create_retrieval_chain` è la chain che orchestra 
# il processo completo di Retrieval Augmented Generation
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [ ]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "Sono interessato a Python, è trattato nei vostri corsi?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]]) # aggiorniamo la storia della conversazione

second_question = "Quanto dura?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=second_question), ai_msg_2["answer"]]) #aggiorniamo la storia della conversazione

print(ai_msg_1["answer"])
print(ai_msg_2["answer"])

Sì, Python è trattato nei corsi offerti da Data Masters. Troverai corsi dedicati alla programmazione in Python, strumenti per l'analisi dei dati con Python e corsi che utilizzano Python per l'analisi e la visualizzazione dei dati. Sono disponibili corsi per principianti e avanzati per soddisfare diverse esigenze.
I corsi Data Masters variano in durata, ma offrono formazione flessibile e pratica in AI, Data Science e Machine Learning. Potrai scegliere tra corsi per principianti e avanzati, studiando dove e quando vuoi. Per informazioni specifiche sulla durata dei corsi, ti consiglio di consultare direttamente il sito web di Data Masters.


In [23]:
ai_msg_3 = rag_chain.invoke({"input": "che assistenza si ha durante lo studio?", "chat_history": chat_history})

print(ai_msg_3)

{'input': 'che assistenza si ha durante lo studio?', 'chat_history': [HumanMessage(content='Sono interessato a Python, è trattato nei vostri corsi?', additional_kwargs={}, response_metadata={}), "Sì, Python è un linguaggio di programmazione molto importante e trattato nei corsi. È considerato uno strumento essenziale per navigare e eccellere in un mondo sempre più guidato dai dati e dall'automazione. La sua semplicità, flessibilità e l'ampia community di sviluppatori lo rendono ideale sia per principianti che per esperti.\n", HumanMessage(content='Quanto dura il corso?', additional_kwargs={}, response_metadata={}), 'La Masterclass ha una durata di 3 mesi. Tutte le lezioni sono accessibili senza limiti e potrai usufruire del pdf con le slide.'], 'context': [Document(id='e6501ee3-4272-4def-bb62-dc9be98bcb2e', metadata={'source': 'https://datamasters.it/blog/guida-completa-a-microsoft-copilot/', 'content_type': 'text/html', 'title': "Copilot: cos'è, a cosa serve e come funziona", 'descrip

In [24]:
ai_msg_3["answer"]

"Durante i corsi è disponibile assistenza continua da parte dei docenti per qualsiasi dubbio o domanda. Puoi contattare direttamente i docenti tramite il canale dedicato all'interno della Community Discord di Data Masters. Tutti i materiali, come slide e video delle lezioni, sono scaricabili e rivedibili.\n"